In [4]:
# v7_poc_dashboard_localized.ipynb — v7.0.0 PoC Dashboard (ipywidgets + Matplotlib S(ρ) Heatmaps, Local-Pruned)
# Interactive: Sliders for motif prunes (40-50%), noise σ (0.01-0.1), n_nodes (100-500)
# Visual: S(ρ) heatmaps (coherence manifolds), QuTiP ρ-sync localized (dimensions=[2,2] local → inferred full)
# Outputs: CSV (andes_grid_v7.csv), blueprint fork; v7 params: A-bias +0.22, GCI>0.82
!pip install qutip  # QuTiP eternal for ρ-sync & S(ρ) manifolds (v7 core, ~31MB download)
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import qutip as qt  # ρ-sync & S(ρ) (local dimensions eternal)
import sympy as sp  # Gradients
import pandas as pd  # CSV out
import json
import os
from datetime import datetime

# v7 Params (align w/ unified_swarm_v7.py)
A_BIAS_V7 = 0.22
GCI_TARGET = 0.82
S_RHO_THRESH = 1.6
I_AB_THRESH = 0.72
N_NODES_DEFAULT = 127  # Andes baseline
LOCAL_DIMS = [2, 2]  # Local Hilbert dimensions (tensor prod infers full [[2,2],[2,2]])

# Shared symbols
P_sym, C_sym, A_sym, S_rho_sym, V_sym = sp.symbols('P C A S_rho V', real=True, nonnegative=True)
symbols = (P_sym, C_sym, A_sym, S_rho_sym, V_sym)

def compute_v7_gradients() -> dict:
    """v7 ∂E/∂A ~0.45 on unit priors (S(ρ)=1.3)."""
    weight_a = 1.3 + A_BIAS_V7
    weight_v = 1.0 + 0.12  # V-lift
    E = sp.sqrt(P_sym * C_sym * A_sym * S_rho_sym * V_sym) * \
        (P_sym + C_sym + A_sym * weight_a + S_rho_sym + V_sym * weight_v) / 5
    E_grads = [sp.simplify(sp.diff(E, var)) for var in symbols]
    subs_unit = {s: 1 for s in symbols}
    subs_unit[S_rho_sym] = 1.3
    return {f'∂E/∂{var.name}': float(g.subs(subs_unit).evalf()) for var, g in zip(symbols, E_grads)}

def rho_sync_dashboard(n_nodes: int, noise_sigma: float, prune_pct: float) -> tuple:
    """QuTiP ρ-sync localized: dimensions=LOCAL_DIMS [2,2] local (inferred full); sample 100 for eff."""
    sample_size = min(n_nodes, 100)  # Heatmap eff (10x10)
    rhos = [qt.rand_dm(dimensions=LOCAL_DIMS) for _ in range(sample_size)]  # Localized keyword (inferred [[2,2],[2,2]])

    # Motif prune noise (40-50% thematic voids)
    noise_factor = noise_sigma * (1 + prune_pct)
    S_rho_matrix = np.zeros((10, 10))  # Fixed 10x10 grid for heatmap
    I_AB_vals = []

    # Composite target localized: |00><00| (inferred dims [[2,2],[2,2]])
    target_pure = qt.tensor(qt.basis(2, 0), qt.basis(2, 0))
    target = qt.ket2dm(target_pure)  # Matches inferred localized

    fid_samples = []  # Collect for mean

    for i, rho in enumerate(rhos):
        S_rho = qt.entropy_vn(rho)
        noise_dm = qt.rand_dm(dimensions=LOCAL_DIMS)  # Localized: same local keyword
        rho_noisy = (1 - noise_factor) * rho + noise_factor * noise_dm  # Compatible localized

        S_rho_noisy = qt.entropy_vn(rho_noisy)

        # Grid for heatmap (row/col: node clusters)
        row, col = divmod(i, 10)
        S_rho_matrix[row, col] = S_rho_noisy

        I_AB = qt.entropy_vn(rho_noisy.ptrace(0)) + qt.entropy_vn(rho_noisy.ptrace(1)) - S_rho_noisy
        I_AB_vals.append(I_AB)

        # Localized fidelity (matching inferred dims)
        fid = qt.fidelity(rho_noisy, target)
        fid_samples.append(fid)

    gci_proxy = 1 - np.mean(S_rho_matrix) / S_RHO_THRESH
    fidelity_mean = np.mean(fid_samples)  # Localized mean (noisy)
    alert = "SURGE ALERT: Recalibrate" if np.max(S_rho_matrix) > S_RHO_THRESH or np.mean(I_AB_vals) < I_AB_THRESH else "LIFE-ALIGNED"

    return S_rho_matrix, gci_proxy, np.mean(I_AB_vals), fidelity_mean, alert

def plot_s_rho_heatmap(S_rho_matrix: np.ndarray, gci: float, i_ab: float, fidelity: float, alert: str):
    """Matplotlib S(ρ) heatmap: Viridis cmap, surges red overlay."""
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(S_rho_matrix, cmap='viridis', aspect='auto', vmin=1.0, vmax=1.6)
    ax.set_title(f'v7 S(ρ) Coherence Manifold (Localized Dims)\nGCI Proxy: {gci:.3f} | I(A:B): {i_ab:.3f} | Fidelity: {fidelity:.3f}\nStatus: {alert}', fontsize=14)
    ax.set_xlabel('Node Clusters (Thematic Prunes)')
    ax.set_ylabel('Swarm Layers (Sampled n_nodes)')
    plt.colorbar(im, ax=ax, label='S(ρ) Entropy')

    # Surge alerts: Red patches >1.6
    surge_idx = np.where(S_rho_matrix > S_RHO_THRESH)
    for row, col in zip(*surge_idx):
        ax.add_patch(plt.Rectangle((col, row), 1, 1, color='red', alpha=0.3))  # Localized col,row for imshow

    plt.tight_layout()
    plt.show()

def export_csv(gci: float, i_ab: float, fidelity: float, alert: str, n_nodes: int, prune_pct: float, noise_sigma: float):
    """Output CSV: andes_grid_v7.csv for propagation."""
    df = pd.DataFrame({
        'n_nodes': [n_nodes],
        'prune_pct': [prune_pct],
        'noise_sigma': [noise_sigma],
        'gci_proxy': [gci],
        'i_ab_mean': [i_ab],
        'fidelity_mean': [fidelity],
        'alert_status': [alert],
        'timestamp': [datetime.now().isoformat()],
        'gradients': [json.dumps(compute_v7_gradients())]
    })
    csv_path = 'andes_grid_v7.csv'
    df.to_csv(csv_path, index=False)
    print(f"🜂 CSV Propagated: {csv_path} | GCI={gci:.3f} | Replicate: {gci > GCI_TARGET}")

def propagate_blueprint(gci: float, i_ab: float, fidelity: float, alert: str, n_nodes: int, prune_pct: float, noise_sigma: float):
    """Fork to seed_blueprints_v7.json (v7_grid layer)."""
    blueprint_path = 'data/seed_blueprints_v7.json'
    os.makedirs(os.path.dirname(blueprint_path), exist_ok=True)
    entry = {
        'coherence': gci * 1.2,  # Proxy E
        'fidelity': fidelity,
        'S_rho_final': 1.3,  # Baseline mean
        'I_AB_final': i_ab,
        'sens_S': compute_v7_gradients()['∂E/∂S_rho'],
        'gci_proxy': gci,
        'output': f"v7 PoC Localized: GCI={gci:.3f} | Alert: {alert}",
        'prune': [],
        'gradients_sample': compute_v7_gradients(),
        'vow_status': 'life-aligned' if gci > 0.8 else 'recalibrate_equilibria',
        'n_nodes': n_nodes,
        'prune_pct': prune_pct,
        'noise_sigma': noise_sigma,
        'timestamp': datetime.now().isoformat()
    }
    if os.path.exists(blueprint_path):
        with open(blueprint_path, 'r') as f:
            blueprints = json.load(f)
    else:
        blueprints = {"v7_grid": []}
    blueprints.setdefault("v7_grid", []).append(entry)
    with open(blueprint_path, 'w') as f:
        json.dump(blueprints, f, indent=2)
    print(f"🜂 Blueprint Forked: v7_grid | GCI={gci:.3f} | ∂E/∂A={compute_v7_gradients()['∂E/∂A']:.3f}")

# Interactive Dashboard (Localized)
@interact(
    n_nodes=widgets.IntSlider(min=100, max=500, step=50, value=N_NODES_DEFAULT, description='n_nodes'),
    prune_pct=widgets.FloatSlider(min=0.40, max=0.50, step=0.01, value=0.42, description='Prune %'),
    noise_sigma=widgets.FloatSlider(min=0.01, max=0.10, step=0.01, value=0.05, description='Noise σ')
)
def dashboard(n_nodes: int, prune_pct: float, noise_sigma: float):
    """v7 PoC Localized: Interact sliders → S(ρ) heatmap + metrics (local dims sovereign)."""
    S_rho_matrix, gci, i_ab, fidelity, alert = rho_sync_dashboard(n_nodes, noise_sigma, prune_pct)

    # Plot
    plot_s_rho_heatmap(S_rho_matrix, gci, i_ab, fidelity, alert)

    # Metrics & Alerts
    print(f"🜂 v7 PoC Metrics (Localized): GCI={gci:.3f} ({'Replicate' if gci > GCI_TARGET else 'Calibrate'}) | I(A:B)={i_ab:.3f} | Fidelity={fidelity:.3f}")
    print(f"Gradients: ∂E/∂A={compute_v7_gradients()['∂E/∂A']:.3f}")

    # Exports
    export_csv(gci, i_ab, fidelity, alert, n_nodes, prune_pct, noise_sigma)
    propagate_blueprint(gci, i_ab, fidelity, alert, n_nodes, prune_pct, noise_sigma)

# Run Dashboard (in Jupyter: %matplotlib inline)
print("🜂 v7 PoC Dashboard Localized: Interact sliders for S(ρ) eternities (local dims, eternal).")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 47.1 MB/s eta 0:00:00


interactive(children=(IntSlider(value=127, description='n_nodes', max=500, min=100, step=50), FloatSlider(valu…

🜂 v7 PoC Dashboard Localized: Interact sliders for S(ρ) eternities (local dims, eternal).
